## Using SerpAPI to webscrape google's showtimes scraper

In [39]:
import re
import requests

### Example

In [29]:
url = 'https://serpapi.com/search.json?q=AMC+Barton+Creek+Square+14&location=Austin%2C+Texas%2C+United+States&hl=en&gl=us&api_key=795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8'

In [30]:
respons = requests.get(url)

In [31]:
respons.json()['showtimes'][0]['movies'][0]

{'name': 'Doctor Strange in the Multiverse of Madness',
 'link': 'https://www.google.com/search?q=&si=AC1wQDAGEQiTPWsvhpPV4-xl97AcUfloRjKp4qat4I8-Kg1HxUZlh2hLOg8aj__svnrQzhOeuRt0XWQO0aQ5e0fgfdZ5L_-DKOa8yWulzvNv7DmICt36IVJV-SCeXHRkJm5uPYvHtnnvHV0ggIt9zeGAwpAdjOYR7j3fHrM0Dpq4qwD5xhOiM0RH5nV40o5n_co7tcOFzsWeZX4vzMK-IJ-FcPoF4paOQ-W6kweqp0GF3nimu3iPjqEocCTwjrfZUttKOz-BaNfPYr2mmh45HkAEBqhIyhigRw%3D%3D&sa=X&ved=2ahUKEwj7qPfg6eD3AhUhl3IEHWAMBe4QxyZ6BAgbEBM',
 'showing': [{'time': ['10:30am',
    '11:00am',
    '11:30am',
    '12:30pm',
    '1:30pm',
    '2:00pm',
    '3:30pm',
    '4:30pm',
    '4:45pm',
    '5:00pm',
    '6:30pm',
    '7:30pm',
    '8:00pm',
    '9:30pm',
    '10:30pm',
    '11:00pm'],
   'type': 'Standard'},
  {'time': ['12:00pm', '3:00pm', '6:00pm', '9:00pm'], 'type': 'IMAX'},
  {'time': ['2:30pm', '5:30pm', '8:30pm'], 'type': '3D'},
  {'time': ['10:00am', '1:00pm', '4:00pm', '7:00pm', '10:00pm'],
   'type': 'Dolby Cinema'}]}

### Applying to Japan

In [32]:
url = 'https://serpapi.com/search.json'

In [33]:
query = 'united cinemas urawa'
query = 'movix saitama'
query = 'toho cinema lalaport fujimi'

In [34]:
params = {
  "q": query, 
  "hl": "en",
  "gl": "us",
  "api_key": "795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8"
}

In [35]:
response = requests.get(url, params=params)

In [36]:
showtimes = response.json()['showtimes']

In [40]:
st=showtimes

In [47]:
def search(query):
    '''
    takes cinema query and returns googles showtimes, a list of dictionaries for each day
    '''
    url = 'https://serpapi.com/search.json'
    params = {"q": query, "hl": "en","gl": "us",
              "api_key": "795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8"}
    
    response = requests.get(url, params=params)
    print(f"Response :{response.status_code}")
    showtimes = response.json()['showtimes']
    
    return showtimes

In [48]:
showtimes = search('united cinemas urawa')

Response :200


### Extracting relevant info

In [ ]:
def showtimes_remove_jp(title_list):
    pattern = r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]' 
    return [title for title in title_list if bool(re.search(pattern, title)) == False]
    

#### Dates collected

##### function

In [ ]:
def start_end(showtimes):
    '''
    takes showtimes dictionary given by googlesearch and returns tuple of startday and endday
    '''
    startday = showtimes[0]['day'] + ' ' + showtimes[0]['date']
    endday   = showtimes[-1]['day'] + ' ' + showtimes[-1]['date']
    
    return (startday,endday)

In [ ]:
start_end(st)

#### List of movies showing each day

In [ ]:
showtimes[0]

In [ ]:
daymovies = {}
for day in showtimes:
    print()
    daymovies[day['day'].lower] = [movie['name'] for movie in day['movies']]


In [ ]:
daymovies = {day['date'].split()[1] : [movie['name'] for movie in day['movies']] for day in showtimes }
#makes a dictionary with each date value as key and a list of movies that day as the value

##### function

In [85]:
def eng_movies_by_day(showtimes):
    '''
    takes showtimes dict and returns a dictionary- keys are each day, values are lists of movie titles
    '''
    pattern = r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]' 
    moviesbyday = {day['date'].split()[1] : [movie['name'] for movie in day['movies'] if bool(re.search(pattern, movie['name'])) == False] for day in showtimes }
    return moviesbyday

def movies_by_day(showtimes):
    '''
    takes showtimes dict and returns a dictionary- keys are each day, values are lists of movie titles
    '''
    moviesbyday = {day['date'].split()[1] : [movie['name'] for movie in day['movies']] for day in showtimes }
    return moviesbyday

In [80]:
def mbd_to_df(mbd):
    maxlength = max([len(movies) for movies in mbd.values()])
    for key in mbd.keys():
        while len(mbd[key]) < maxlength:
            mbd[key].append('-')
    return pd.DataFrame(mbd)

In [81]:
mbd_to_df(movies_by_day(showtimes))

,15,16,17,18,19
0,Shin Ultraman,Shin Ultraman,Shin Ultraman,Shin Ultraman,Shin Ultraman
1,Detective Conan: The Bride of Halloween,Detective Conan: The Bride of Halloween,Detective Conan: The Bride of Halloween,Detective Conan: The Bride of Halloween,Detective Conan: The Bride of Halloween
2,Doctor Strange in the Multiverse of Madness,Doctor Strange in the Multiverse of Madness,Doctor Strange in the Multiverse of Madness,Doctor Strange in the Multiverse of Madness,Doctor Strange in the Multiverse of Madness
3,Bubble,Bubble,Bubble,Bubble,Bubble
4,Fantastic Beasts: The Secrets of Dumbledore,Fantastic Beasts: The Secrets of Dumbledore,Fantastic Beasts: The Secrets of Dumbledore,Fantastic Beasts: The Secrets of Dumbledore,Fantastic Beasts: The Secrets of Dumbledore
5,Free! – the Final Stroke – the first volume,Crayon Shin-chan: Mononoke Ninja Chinpūden,Crayon Shin-chan: Mononoke Ninja Chinpūden,Crayon Shin-chan: Mononoke Ninja Chinpūden,Crayon Shin-chan: Mononoke Ninja Chinpūden
6,Jujutsu Kaisen 0,Free! – the Final Stroke – the first volume,Free! – the Final Stroke – the first volume,Free! – the Final Stroke – the first volume,Free! – the Final Stroke – the first volume
7,-,C'mon C'mon,C'mon C'mon,C'mon C'mon,C'mon C'mon


#### list of all movies showing

In [ ]:
movie_set = {movie['name'] for movie in day['movies'] for day in showtimes}

In [ ]:
{movie['name'] for movie in day['movies'] for day in showtimes}

In [ ]:
list(movie_set)

In [90]:
def movie_list(showtimes):
    '''
    goes through showtimes and returns a list of each unique movie
    '''
    movie_set = {movie['name'] for day in showtimes for movie in day['movies']}
    return list(movie_set)

def eng_movie_list(showtimes):
    '''
    goes through showtimes and returns a list of each unique movie
    '''
    pattern = r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]'
    movie_set = {movie['name'] for day in showtimes for movie in day['movies'] if bool(re.search(pattern, movie['name'])) == False}
    return list(movie_set)

In [91]:
movie_list(showtimes)

['Free! – the Final Stroke – the first volume',
 'Shin Ultraman',
 "C'mon C'mon",
 '劇場版ラジエーションハウス',
 '流浪の月',
 'Detective Conan: The Bride of Halloween',
 'Doctor Strange in the Multiverse of Madness',
 'Jujutsu Kaisen 0',
 '映画 おそ松さん',
 'Fantastic Beasts: The Secrets of Dumbledore',
 'Bubble',
 'ホリック xxxHOLiC',
 'Crayon Shin-chan: Mononoke Ninja Chinpūden']

In [92]:
eng_movie_list(showtimes)

['Free! – the Final Stroke – the first volume',
 'Shin Ultraman',
 "C'mon C'mon",
 'Detective Conan: The Bride of Halloween',
 'Doctor Strange in the Multiverse of Madness',
 'Jujutsu Kaisen 0',
 'Fantastic Beasts: The Secrets of Dumbledore',
 'Bubble',
 'Crayon Shin-chan: Mononoke Ninja Chinpūden']

Daily Showtimes DF

In [ ]:
def daily_df_list(showtimes):
    

In [152]:
i=0
showtimes[0]['day'] +' '+ showtimes[0]['date']


'Sun May 15'

In [192]:
day = showtimes[0]

In [336]:
showtimes[2]['movies'][0]['showing'][0]['time'] = '8:15am'+'9:10am'+'10:30am'+'1:00pm'+'3:30pm'+'5:55pm'+'7:00pm'+'8:25pm'+'8:55pm'

In [327]:
def eng_day_to_df(day):
    pattern = r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]'
    daydict=[{'name':movie['name'],'times':movie['showing']} for movie in day['movies'] if bool(re.search(pattern, movie['name'])) == False]
    for movie in daydict:
        movie['Movie'] = movie.pop('name')
        for typedict in movie['times']:
            movie[typedict['type']] = typedict['time']
        movie.pop('times')
    daydf = pd.DataFrame(daydict).set_index('Movie').fillna('-')
    return daydf

def day_to_df(day):
    daydict=[{'name':movie['name'],'times':movie['showing']} for movie in day['movies']]
    for movie in daydict:
        movie['Movie'] = movie.pop('name')
        for typedict in movie['times']:
            movie[typedict['type']] = typedict['time']
        movie.pop('times')
    daydf = pd.DataFrame(daydict).set_index('Movie').fillna('-')
    return daydf

def eng_daily_df_list(showtimes):
    return [eng_day_to_df(day) for day in showtimes]

def daily_df_list(showtimes):
    return [day_to_df(day) for day in showtimes]

In [318]:
eng_day_to_df(day)

,Standard,IMAX,IMAX 3D
Movie,,,
Shin Ultraman,"[2:45pm, 3:30pm, 5:55pm, 7:00pm, 8:25pm]","[4:25pm, 9:35pm]",-
Detective Conan: The Bride of Halloween,"[3:50pm, 6:15pm, 8:40pm]",-,-
Doctor Strange in the Multiverse of Madness,"[2:55pm, 5:35pm, 8:15pm]",-,[6:50pm]
Bubble,"[3:45pm, 8:55pm]",-,-
Fantastic Beasts: The Secrets of Dumbledore,[3:40pm],-,-
Free! – the Final Stroke – the first volume,[6:35pm],-,-
Jujutsu Kaisen 0,[9:25pm],-,-


In [337]:
eng_daily_df_list(showtimes)[2]

,Standard,IMAX,IMAX 3D
Movie,,,
Shin Ultraman,8:15am9:10am10:30am1:00pm3:30pm5:55pm7:00pm8:2...,"[8:40am, 1:55pm, 4:25pm, 9:35pm]",-
Detective Conan: The Bride of Halloween,"[10:55am, 1:25pm, 3:50pm, 6:15pm, 8:40pm]",-,-
Doctor Strange in the Multiverse of Madness,"[10:20am, 2:55pm]",-,"[11:10am, 6:50pm]"
Bubble,"[8:10am, 10:20am, 1:00pm, 3:50pm, 6:00pm, 9:25pm]",-,-
Fantastic Beasts: The Secrets of Dumbledore,"[11:40am, 3:40pm]",-,-
Crayon Shin-chan: Mononoke Ninja Chinpūden,"[8:10am, 1:45pm]",-,-
Free! – the Final Stroke – the first volume,"[8:30am, 6:35pm]",-,-
C'mon C'mon,[3:05pm],-,-


In [277]:
ddf.fillna('-', inplace=True)

In [278]:
ddf


,Standard,IMAX,IMAX 3D
Movie,,,
Shin Ultraman,"[2:45pm, 3:30pm, 5:55pm, 7:00pm, 8:25pm]","[4:25pm, 9:35pm]",-
Detective Conan: The Bride of Halloween,"[3:50pm, 6:15pm, 8:40pm]",-,-
Doctor Strange in the Multiverse of Madness,"[2:55pm, 5:35pm, 8:15pm]",-,[6:50pm]
Bubble,"[3:45pm, 8:55pm]",-,-
Fantastic Beasts: The Secrets of Dumbledore,[3:40pm],-,-
Free! – the Final Stroke – the first volume,[6:35pm],-,-
Jujutsu Kaisen 0,[9:25pm],-,-
